# 4.8: List of Contents (Task Questions)

### Question 2: Aggregated Mean of "order_number" grouped by "department_id"
### Question 3: Results of Question 2
### Question 4: Creating Loyalty Flag
### Question 5: Spending Habits by Loyalty Flag
### Question 6: High vs. Low Spenders
### Question 7: Creating Frequency Flag
### Small Data Correction: Renaming of two flags
### Question 9: Export Dataframe


In [1]:
# Importing Libraries

# Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'/Users/tristansavella/Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_task47.pkl'))

In [3]:
df_department = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'))

### Question 2: Aggregated Mean of "order_number" grouped by "department_id"
In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [4]:
ords_prods_merge.groupby('product_name')

In [5]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.213779
6,16.439806
7,17.225773
8,15.340520


### Question 3: Results of Question 2
Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

The subset did not have purchases from all 21 departments. The range of mean values is also wider in the subset.

### Question 4: Creating Loyalty Flag
Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [6]:
#Create max_order to use for flagging
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
#To see all rows
pd.options.display.max_rows = None

In [8]:
#Assigning Loyalty Flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10,'loyalty_flag'] = 'New customer'

In [9]:
#Checking value counts
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15891077
Loyal customer      10293737
New customer         6249398
Name: count, dtype: int64

I did do a head check to see if the columns were intact to view the first 100 entries, but deleted for size

### Question 5: Spending Habits by Loyalty Flag
#### The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

New customers spend the most by average, followed by regular customer. Loyal Customers spend the least, indicating that that those most loyal may prefer to look for good deals.

In [10]:
#Group by Prices
ords_prods_merge.groupby('prices')

In [11]:
#Mean Price spent by Each Loyalty Group Level
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,10.388747
New customer,13.294370
Regular customer,12.496203


### Question 6: High vs. Low Spenders

#### The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”

If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [12]:
#Create mean_price to use for later flagging high and low spenders
ords_prods_merge['mean_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [13]:
#Assigning 'spend_flag' - this column currently appears as 'type of spender' for some reason and was renamed to 'spending_flag' at the end of this notebook
ords_prods_merge.loc[ords_prods_merge['mean_price'] >= 10, 'spend_flag'] = 'High spender'
ords_prods_merge.loc[ords_prods_merge['mean_price'] < 10, 'spend_flag'] = 'Low spender'

In [14]:
#checking value count for each high and low spenders
ords_prods_merge['spend_flag'].value_counts()

spend_flag
Low spender     31798751
High spender      635461
Name: count, dtype: int64

Please Note: I did checks for both of these steps but deleted them due to notebook size

### Question 7: Creating Frequency Flag

#### In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”

If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”


In [15]:
#create median_days_passed to use for flagging customer frequency
ords_prods_merge['median_days_passed'] = ords_prods_merge.groupby(['user_id'])['days_passed_since_last_order'].transform(np.median)

In [16]:
ords_prods_merge.head(5)

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_passed_since_last_order,product_id,add_to_cart_order,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,type_of_spender,spend_flag,median_days_passed
0,0,2539329,1,prior,1,2,8,NaN,196,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,Low spender,20.5
1,1,2398795,1,prior,2,3,7,15.0,196,1,...,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,Low spender,20.5
2,2,473747,1,prior,3,3,12,21.0,196,1,...,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,Low spender,20.5
3,3,2254736,1,prior,4,4,7,29.0,196,1,...,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,Low spender,20.5
4,4,431534,1,prior,5,4,15,28.0,196,1,...,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,Low spender,20.5


In [17]:
#assigning spend_flag; this column was renamed to 'frequency_flag' at the end of this notebook
ords_prods_merge.loc[ords_prods_merge['median_days_passed'] > 20, 'spend_flag'] = 'Non-Frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_passed'] <= 20) & (ords_prods_merge['median_days_passed'] > 10), 'spend_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_passed'] <= 10, 'spend_flag'] = 'Frequent customer'

In [18]:
ords_prods_merge.head(20)

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_passed_since_last_order,product_id,add_to_cart_order,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,type_of_spender,spend_flag,median_days_passed
0,0,2539329,1,prior,1,2,8,NaN,196,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
1,1,2398795,1,prior,2,3,7,15.0,196,1,...,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
2,2,473747,1,prior,3,3,12,21.0,196,1,...,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
3,3,2254736,1,prior,4,4,7,29.0,196,1,...,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
4,4,431534,1,prior,5,4,15,28.0,196,1,...,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
5,5,3367565,1,prior,6,2,7,19.0,196,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
6,6,550135,1,prior,7,1,9,20.0,196,1,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
7,7,3108588,1,prior,8,1,14,14.0,196,2,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
8,8,2295261,1,prior,9,1,16,0.0,196,4,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
9,9,2550362,1,prior,10,4,8,30.0,196,1,...,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5


### Small Data Correction: Renaming of two flags

In [19]:
# Renaming Spend Flag
ords_prods_merge = ords_prods_merge.rename(columns={'spend_flag':'Frequency_Flag'})

In [20]:
# Renaming "Type_of_Spender"
ords_prods_merge = ords_prods_merge.rename(columns={'type_of_spender':'Spending_Flag'})

In [21]:
ords_prods_merge.head(20)

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_passed_since_last_order,product_id,add_to_cart_order,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,Spending_Flag,Frequency_Flag,median_days_passed
0,0,2539329,1,prior,1,2,8,NaN,196,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
1,1,2398795,1,prior,2,3,7,15.0,196,1,...,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
2,2,473747,1,prior,3,3,12,21.0,196,1,...,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
3,3,2254736,1,prior,4,4,7,29.0,196,1,...,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
4,4,431534,1,prior,5,4,15,28.0,196,1,...,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
5,5,3367565,1,prior,6,2,7,19.0,196,1,...,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
6,6,550135,1,prior,7,1,9,20.0,196,1,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
7,7,3108588,1,prior,8,1,14,14.0,196,2,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
8,8,2295261,1,prior,9,1,16,0.0,196,4,...,Mid-range product,Regularly busy,Busiest days,Most orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5
9,9,2550362,1,prior,10,4,8,30.0,196,1,...,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,Non-Frequent customer,20.5


### Question 9: Export Dataframe
Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [22]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_task48.pkl'))

In [23]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,10.388747
New customer,13.294370
Regular customer,12.496203
